In [1]:
import sys
sys.path.append('/home1/yxiao977/labwork')
import test_func 

In [2]:
request = {
    'hic_path':'/home1/yxiao977/sc1/train_akita/data/5000res_0.5thres_hic_filter_both_bin.cool',
    'cis_total_path':'/home1/yxiao977/sc1/train_akita/data/5000res_bins_cistotal_ratio.bed'
}

test_func.test_cis_total_filter(request)

2023-10-01 13:30:05,743 - one - DEBUG : Start to find bad bins
2023-10-01 13:30:05,771 - one - DEBUG : Start to find good bins
2023-10-01 13:30:05,827 - one - DEBUG : Start to create pixel dataframe based on binid1
2023-10-01 13:35:20,276 - one - DEBUG : Start to create pixel dataframe based on binid2
2023-10-01 13:53:01,865 - one - DEBUG : Start to sum up counts
2023-10-01 13:53:02,538 - one - INFO : Pass bin1 checkpoint
2023-10-01 13:53:16,178 - one - INFO : Pass bin2 checkpoint


In [1]:
### Test need to delete later
import cooler
import numpy as np
clr = cooler.Cooler('/home1/yxiao977/sc1/train_akita/data/5000res_0.5thres_hic_filter_both_bin.mcool::resolutions/5000')
chunks = clr.bins()[:10]
pixels = clr.pixels().fetch('chr1_pilon:0-500000')

In [42]:
import numpy as np
import pandas as pd
import cooltools
import cooler
import logging
def _bins_cis_total_ratio_filter(clr, thres):
    """
    Filter out bins with low cis-to-total coverage ratio from a Cooler object.

    Parameters
    ----------
    clr : cooler.Cooler
        A Cooler object containing Hi-C contact matrices.
    thres : float
        The threshold cis-to-total coverage ratio below which bins are considered bad.

    Returns
    -------
    numpy.ndarray
        An array of bin indices with low cis-to-total coverage ratio.

    Notes
    -----
    This function uses the `cooltools.coverage` function to compute the cis-to-total
    coverage ratio for each bin, and returns an array of bin indices with a ratio
    below `thres`. The returned array contains the indices of bins with low cis-to-total
    coverage ratio, which can be used to filter out these bins from the input Cooler object.

    """
    coverage = cooltools.coverage(clr)
    cis_total_cov = coverage[0] / coverage[1]
    bins_mask = np.argwhere((cis_total_cov <= thres)).reshape(-1)

    return bins_mask

bin_mask = _bins_cis_total_ratio_filter(clr, 0.5)

/tmp/SLURM_16713091/ipykernel_42742/176519719.py:31: RuntimeWarning: invalid value encountered in true_divide
  cis_total_cov = coverage[0] / coverage[1]


In [6]:
chunk_pixels = clr.pixels()[10_000_000 * 0 : 10_000_000 * 1]


In [61]:
def _write_filtered_pixels_files(clr, chunk_pixels, bin_mask, count_output_path):
    """
    Write the counts of interactions between genomic regions to a text file.

    This function takes a cooler object containing the genomic regions, a DataFrame containing the counts of interactions
    between genomic regions, and the path to the output text file. It then writes the counts of interactions to the output
    text file.

    Parameters
    ----------
    clr : cooler.Cooler
        A cooler object containing the genomic regions.
    chunk_pixels : pandas.DataFrame
        A DataFrame containing the counts of interactions between genomic regions.
    count_output_path : str
        The path to the output text file.

    Returns
    -------
    None
    
    """
    bad_bins_index = np.array(range(clr.bins().shape[0]))[bin_mask]
    pixels_mask = chunk_pixels['bin1_id'].isin(bad_bins_index)
    chunk_pixels.loc[pixels_mask, 'count'] = 0
    pixels_mask = chunk_pixels['bin2_id'].isin(bad_bins_index)
    chunk_pixels.loc[pixels_mask, 'count'] = 0
    # Then write the counts in text file:
    with open(count_output_path, 'a+') as count_file:
        for i, row in chunk_pixels.iterrows():
            bin1_id, bin2_id, count = row.value()
            chrom1, start1, end1 = list(clr.bins()[bin1_id])[:3]
            chrom2, start2, end2 = list(clr.bins()[bin2_id])[:3]

            count_file.write(f"{chrom1}\t{start1}\t{end1}\t{chrom2}\t{start2}\t{end2}\t{count}\n")

In [62]:
def filter_pixels_by_masked_bin(clr, thres, chromsize_output_path, count_output_path, chunksize=10_000_000):
    """
    Filter the pixels of a cooler object based on a binary mask of masked bins.

    Parameters
    ----------
    clr : cooler.Cooler
        The cooler object to filter.
    thres : float
        The threshold for the cis-total ratio filter.
    chromsize_output_path : str
        The path to the output file where the chromosome sizes will be written.
    count_output_path : str
        The path to the output file where the filtered pixels will be written.
    chunksize : int, optional
        The size of the chunks to process the pixels in, in number of pixels.

    Returns
    -------
    None
    
    """

    logger.debug('Start to make bin mask...')
    bin_mask = _bins_cis_total_ratio_filter(clr, thres)
    tot_pixels = clr.pixels().shape[0]
    iter_num = tot_pixels // chunksize

    # First write the chromosome sizes:
    logger.debug('Start to create chromsize file...')
    with open(chromsize_output_path, 'a+') as chromsize_file:
        for i, chromsize in enumerate(clr.chromsizes):
            chromsize_file.write(f"{clr.chromnames[i]}\t{chromsize}\n")

    logger.debug('Start to create pixels counts file...')
    for i in range(iter_num):
        chunk_pixels = clr.pixels()[chunksize * i : chunksize * (i + 1)]
        
        ### Here we might use multiprocessing to boost the speed, but does order matter###
        _write_filtered_pixels_files(clr, chunk_pixels, bin_mask, count_output_path)
    
    # remainder 
    chunk_pixels = clr.pixels()[chunksize * iter_num:]
    _write_filtered_pixels_files(clr, chunk_pixels, count_output_path)

In [65]:
import logging
logger = logging.getLogger('one')
logger.setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s : %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

thres = 0.5
chromsize_output_path = '/home1/yxiao977/sc1/test_func/chromsize.txt' 
count_output_path = '/home1/yxiao977/sc1/test_func/counts.txt' 
filter_pixels_by_masked_bin(clr, thres, chromsize_output_path, count_output_path, chunksize=10_000_000)

2023-10-03 15:08:34,286 - one - DEBUG : Start to make bin mask...
2023-10-03 15:08:34,286 - one - DEBUG : Start to make bin mask...
DEBUG:one:Start to make bin mask...
/tmp/SLURM_16713091/ipykernel_42742/176519719.py:31: RuntimeWarning: invalid value encountered in true_divide
  cis_total_cov = coverage[0] / coverage[1]
2023-10-03 15:09:20,050 - one - DEBUG : Start to create chromsize file...
2023-10-03 15:09:20,050 - one - DEBUG : Start to create chromsize file...
DEBUG:one:Start to create chromsize file...
2023-10-03 15:09:20,213 - one - DEBUG : Start to create pixels counts file...
2023-10-03 15:09:20,213 - one - DEBUG : Start to create pixels counts file...
DEBUG:one:Start to create pixels counts file...


AttributeError: 'Series' object has no attribute 'value'

In [50]:
bin_index = np.array(range(clr.bins().shape[0]))[bin_mask]

In [68]:
for i, row in chunk_pixels.iterrows():
            bin1_id, bin2_id, count = row
            print(row)
            break

bin1_id    0
bin2_id    0
count      0
Name: 0, dtype: int64


In [16]:
pixels_mask = [bin_mask[i] for i in chunk_pixels['bin1_id'].values]

In [43]:
pixels_mask2 = chunk_pixels['bin1_id'].isin(bin_mask)

In [53]:
pixels_mask[0] = True

In [56]:
chunk_pixels.loc[pixels_mask, 'count']

0    0
Name: count, dtype: int32

In [39]:
np.sum(pixels_mask2)

10116

In [41]:
len(chunk_pixels['bin1_id'].unique())

1513

In [27]:
for i in chunk_pixels['bin1_id'].values:
    if bin_mask[i] == False:
        print(i)
        break

0


In [31]:
for i, value in enumerate(pixels_mask2):
    if value == True:
        print(i)
        break

0


In [34]:
chunk_pixels.iloc[0]

bin1_id       0
bin2_id       0
count      3388
Name: 0, dtype: int64

In [36]:
np.sum(bin_mask)

142

In [28]:
pixels_mask2 = chunk_pixels['bin1_id'].isin(bin_mask)

In [6]:
masked_bin_table = clr.bins()[:]


In [53]:
masked_bin_table.loc[test_arry,'weight'] = 2

In [ ]:
for i in  pixels['bin1_id'].values:
    print(i)

In [50]:
test_arry = np.argwhere((np.array([0, 1, 0, 1]) <= 0.5)).reshape(-1)

In [55]:
import numpy as np

# Sample data
data = np.array([1, 2, 3, 4, 5, 6])

# Create a mask to select even numbers
mask = data % 2 == 0

# Use the mask to filter the data
filtered_data = data[mask]

print(filtered_data)  # Output: [2 4 6]


[2 4 6]
